# Welcome to the COM-304 SDR Tutorials Series
* In this Jupyter Notebook, we are going to see how we can implement the same GNU Radio Transmitter / Receiver in Python

Let's first import the various dependencies that GNU Radio requires

In [1]:
import sys
import signal
import osmosdr
import time

from packaging.version import Version as StrictVersion
from PyQt5 import Qt
from gnuradio import qtgui
from gnuradio import analog
from gnuradio import audio
from gnuradio import blocks
from gnuradio import filter
from gnuradio.filter import firdes
from gnuradio import gr
from gnuradio.fft import window
from gnuradio.eng_arg import eng_float, intx
from argparse import ArgumentParser

In [2]:
class firstFmTransceiver(gr.top_block, Qt.QWidget):

    def __init__(self):
        gr.top_block.__init__(self, "My First FM Transceiver", catch_exceptions=True)

        ##################################################
        # Variables - define your varaibles here :)
        ##################################################
        self.samp_rate = samp_rate = 3.5e6

        ##################################################
        # Blocks
        ##################################################

        self.rational_resampler = filter.rational_resampler_ccc(
                interpolation=100,
                decimation=1,
                taps=[],
                fractional_bw=0)
        self.bladerf = osmosdr.source(args="numchan=" + str(1) + " " + 'bladerf=0')

        self.bladerf.set_time_unknown_pps(osmosdr.time_spec_t())
        self.bladerf.set_sample_rate(samp_rate) # Our bladeRF's sample rate
        self.bladerf.set_center_freq(866e6, 0) # Our bladeRF's Rx Frequency
        self.bladerf.set_freq_corr(0, 0)
        self.bladerf.set_dc_offset_mode(0, 0)
        self.bladerf.set_iq_balance_mode(0, 0)
        self.bladerf.set_gain_mode(False, 0)
        self.bladerf.set_gain(10, 0)
        self.bladerf.set_if_gain(20, 0)
        self.bladerf.set_bb_gain(20, 0)
        self.bladerf.set_antenna('', 0)
        self.bladerf.set_bandwidth(0, 0)
        self.osmosdr_sink_0 = osmosdr.sink(
            args="numchan=" + str(1) + " " + 'bladerf=0'
        )
        self.osmosdr_sink_0.set_time_unknown_pps(osmosdr.time_spec_t())
        self.osmosdr_sink_0.set_sample_rate(samp_rate)
        self.osmosdr_sink_0.set_center_freq(866e6, 0)
        self.osmosdr_sink_0.set_freq_corr(0, 0)
        self.osmosdr_sink_0.set_gain(10, 0)
        self.osmosdr_sink_0.set_if_gain(20, 0)
        self.osmosdr_sink_0.set_bb_gain(20, 0)
        self.osmosdr_sink_0.set_antenna('', 0)
        self.osmosdr_sink_0.set_bandwidth(0, 0)
        self.low_pass_filter_0 = filter.fir_filter_ccf(
            70,
            firdes.low_pass(
                1,
                samp_rate,
                4000,
                1000,
                window.WIN_HAMMING,
                6.76))
        self.blocks_wavfile_source_0 = blocks.wavfile_source('/Users/admin/Desktop/commsproject/commsproject/audio/audio1.wav', True) # Our WAV source file
        self.audio_sink_0 = audio.sink(48000, '', True)
        self.analog_nbfm_tx_0 = analog.nbfm_tx(
        	audio_rate=48000,
        	quad_rate=48000,
        	tau=(75e-6),
        	max_dev=5e3,
        	fh=(-1.0),
                )
        self.analog_nbfm_rx_0 = analog.nbfm_rx(
        	audio_rate=48000,
        	quad_rate=48000,
        	tau=(75e-6),
        	max_dev=5e3,
          )


        ##################################################
        # Connections
        ##################################################
        self.connect((self.analog_nbfm_rx_0, 0), (self.audio_sink_0, 0)) # connect FM receiver to audio sink (PC's speakers)

        self.connect((self.analog_nbfm_tx_0, 0), (self.rational_resampler, 0)) # connect FM transmitter to rational resampler

        self.connect((self.blocks_wavfile_source_0, 0), (self.analog_nbfm_tx_0, 0)) # connect WAV file source to FM trasnmitter

        self.connect((self.low_pass_filter_0, 0), (self.analog_nbfm_rx_0, 0)) # connect low-pass filter to the FM receiver

        self.connect((self.bladerf, 0), (self.low_pass_filter_0, 0)) # connect the bladerf source to the low pass filter

        self.connect((self.rational_resampler, 0), (self.osmosdr_sink_0, 0)) # connect the rational resampler to the bladeRF sink (for TX)


    def closeEvent(self, event):
        self.settings = Qt.QSettings("GNU Radio", "firstFmTransceiver")
        self.settings.setValue("geometry", self.saveGeometry())
        self.stop()
        self.wait()

        event.accept()

    def get_samp_rate(self):
        return self.samp_rate

    def set_samp_rate(self, samp_rate):
        self.samp_rate = samp_rate
        self.low_pass_filter_0.set_taps(firdes.low_pass(1, self.samp_rate, 4000, 1000, window.WIN_HAMMING, 6.76))
        self.osmosdr_sink_0.set_sample_rate(self.samp_rate)
        self.bladerf.set_sample_rate(self.samp_rate)

In [3]:
def main(top_block_cls=firstFmTransceiver, options=None):

    if StrictVersion("4.5.0") <= StrictVersion(Qt.qVersion()) < StrictVersion("5.0.0"):
        style = gr.prefs().get_string('qtgui', 'style', 'raster')
        Qt.QApplication.setGraphicsSystem(style)
    qapp = Qt.QApplication(sys.argv)

    tb = top_block_cls()

    tb.start()

    tb.show()

    def sig_handler(sig=None, frame=None):
        tb.stop()
        tb.wait()

        Qt.QApplication.quit()

    signal.signal(signal.SIGINT, sig_handler)
    signal.signal(signal.SIGTERM, sig_handler)

    timer = Qt.QTimer()
    timer.start(500)
    timer.timeout.connect(lambda: None)

    qapp.exec_()

if __name__ == '__main__':
    main()


[bladeRF common] init: DEBUG: entering initialization
[bladeRF source] init: Opening Nuand bladeRF with device identifier string '*:instance=0'
[bladeRF source] Device: Nuand bladeRF 2.0 Serial # 08b7...8047 FW v2.4.0 FPGA v0.15.0
[bladeRF source] init: Buffers: 512, samples per buffer: 4096, active transfers: 32
[bladeRF source] bladerf_source_c: DEBUG: initialization complete


gr-osmosdr 0.2.0.0 (0.2.0) gnuradio 3.10.7.0
built-in source types: file rtl rtl_tcp uhd hackrf bladerf rfspace airspy airspyhf soapy redpitaya 
gr-osmosdr 0.2.0.0 (0.2.0) gnuradio 3.10.7.0
built-in sink types: 

[bladeRF common] init: DEBUG: entering initialization
[bladeRF sink] init: Opening Nuand bladeRF with device identifier string '*:instance=0'
[bladeRF sink] Device: Nuand bladeRF 2.0 Serial # 08b7...8047 FW v2.4.0 FPGA v0.15.0
[bladeRF sink] init: Buffers: 512, samples per buffer: 4096, active transfers: 32
[bladeRF sink] bladerf_sink_c: DEBUG: initialization complete
len(audio_taps) = 231
[bladeRF source] start: DEBUG: starting source
[bladeRF sink] start: DEBUG: starting sink


uhd hackrf bladerf soapy redpitaya file 
[WARNING @ host/libraries/libbladeRF/src/board/bladerf2/rfic_host.c:523] _rfic_host_get_gain_mode: automatic gain control not valid for TX channels


RuntimeError: super-class __init__() of type firstFmTransceiver was never called

: 